# Part I

In [ ]:
# =============================================================================
# Import libraries
# =============================================================================

import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

print('Libraries imported.')


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Fetching package metadata .............
Solving package specifications: 

In [ ]:
# =============================================================================
# Scrape from Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
# =============================================================================

wikipedia_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(wikipedia_page.content, 'lxml')

wikipedia_table = soup.find('table', class_ = 'wikitable sortable')

header = wikipedia_table.tbody.find('tr')
column_names = ['Postcode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(columns=column_names)

for tr in header.find_next_siblings('tr'):
    tds = tr.find_all('td')
    df = df.append({'Postcode': tds[0].text,
                    'Borough': tds[1].text,
                    'Neighbourhood': tds[2].text}, ignore_index=True)


In [ ]:
df.head

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [ ]:
df['Neighbourhood'] = df['Neighbourhood'].apply(lambda x: x.strip())
df = df[df['Borough']!='Not assigned']

In [ ]:
df

# Combined neighborhoods into one row separated with a comma

In [ ]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [ ]:
df.loc[df['Neighbourhood']=='Not assigned', 'Neighbourhood'] = df['Borough']
df

# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [ ]:
df.shape

# Part II

In [ ]:
df_geo = pd.read_csv ('http://cocl.us/Geospatial_data')
df_geo

In [ ]:
print(df.columns)
print(df_geo.columns)

df.set_index('Postcode')
df_geo.set_index('Postal Code')
df2 = df.join(df_geo).drop('Postal Code', 1)
df2

In [ ]:
df2.shape